In [13]:
#remove cell visibility
from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

## Krmiljenje tempomata vozila

Dinamiko hitrosti vozila, lahko modeliramo (gl. poglavje *Modeliranje dinamike hitrosti vozila* v spremljajočem učbeniku tečaja ICCT za celotno izpeljavo) na naslednji način:

$$
\begin{cases}
\dot{x}=\underbrace{\begin{bmatrix}-\frac{b}{m}&\frac{\tau_{\text{max}}\eta}{mr}\\0&-\frac{1}{2}\end{bmatrix}}_{A}x+\underbrace{\begin{bmatrix}0\\\frac{1}{2}\end{bmatrix}}_{B}t_r \\
y = \underbrace{\begin{bmatrix}3.6&0\end{bmatrix}}_{C}x
\end{cases}
$$

kjer so $y$ hitrost vozila (v km/h), $m$ masa vozila ($1000$ kg), $\tau_{\text{max}}$ maksimalni navor motorja (150 Nm), $r$ radij koles (25 cm) (s konstantnim prestavnim razmerjem $\eta$: 4:1 motor:kolo), $b$ koeficient zraćega upora ($60$ Ns/m), in $\tau_{\%}$ predstavlja lego stopalke za plin (vrednost mora biti med 0 in 1). Sistem je vodljiv in spoznaven.

Cilj je načrtovati krmilni sistem, ki krmili hitrost vozila na način, da deluje prek stopalke za plin na motor, ter izpolnjuje naslednje zahteve:
- brez odstopka v stacionarnem stanju v odzivu na zahtevano spremembo hitrosti v obliki koračne funkcije,
- brez prenihaja, ali pa vsaj s prenihajem manjšim od 1%,
- čas ustalitve krajši od 4 s (dosežena vrednost izhoda naj se razlikuje od tiste v stacionarnem stanju za 5%).

### Načrtovanje regulatorja
Pomembno je poudariti, da ni možno zagotoviti zahteve glede časa ustalitve za vse možne referenčne vrednosti hitrosti. To lahko razložimo s pomočjo fizikalnega ozadja problema: zračni upor je precej večji pri višjih hitrostih, zato vozilo pri teh hitrostih ne more pospeševati tako, kot pospešuje pri nižjih hitrostih. Dodatno lahko pokažemo, da če je zračni upor zanemarljiv  ($b=0$), je maksimalna sprememba hitrosti pri $\tau_{\%} = 1$ enaka 43.2 km/h v 5 s.
Iz napisanega sledi, da bomo krmilni sistem zasnovali tako, da bo izpolnil prvi dve zahtevi in bo njegovo delovanje čim hitrejše.

#### Načrtovanje krmilnika povratne zveze stanj

Za doseg ničelnega odstopka v stacionarnem stanju v odzivu na koračno funkcijo, razširimo sistem z dodatno spremenljivko stanja $x_3 = v-v_{\text{ref}}$, kjer je $v_{\text{ref}}$ referenčni vhod v km/h. Razširjen sistem lahko zapišemo z naslednjo enačbo:

$$
\begin{cases}
\dot{x_a}=\underbrace{\begin{bmatrix}-\frac{b}{m}&\frac{\tau_{\text{max}}\eta}{mr}&0\\0&-\frac{1}{2}&0\\3.6&0&0\end{bmatrix}}_{A_a}x_a+\underbrace{\begin{bmatrix}0\\\frac{1}{2}\\0\end{bmatrix}}_{B_a}t_r+\underbrace{\begin{bmatrix}0\\0\\-1\end{bmatrix}}_{B_{\text{ref}}}v_{\text{ref}} \\
y_a = \underbrace{\begin{bmatrix}3.6&0&0\\0&0&1\end{bmatrix}}_{C_a}x_a
\end{cases}
$$

Opazimo lahko, da je sistem še vedno vodljiv.

Možna rešitev je, da nastavimo lastne vrednosti matrike na vrednosti $-0.06$, $-2.43-2.41i$ in $-2.43+2.41i$ z $K_a=\begin{bmatrix}9.75&8.73&0.15\end{bmatrix}$.

#### Načrtovanje spoznavalnika

Spoznavalnik razvijemo na podlagi začetnega sistema, z namenom, da ocenimo $x_1$ in $x_2$. Pola dinamika napake ($A-LC$) ležita v $-10$ rad/s z $L=\begin{bmatrix}5.4&10.45\end{bmatrix}^T$.

Sistem s krmilnikom je simuliran spodaj. Možno ga je simulirati tudi z nasičenjem vhoda.

In [14]:
%matplotlib inline
import control as control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [15]:
# Preparatory cell

Aa = numpy.matrix([[-3/50,2.4,0],[0,-0.5,0],[3.6,0,0]])
Ba = numpy.matrix('0; 0.5; 0')
A = numpy.matrix([[-3/50, 2.4],[0,-0.5]])
B = numpy.matrix('0; 0.5')
C = numpy.matrix('3.6 0')
X0 = numpy.matrix('0.; 0.')
K = numpy.matrix([9.75,8.73,0.15])
L = numpy.matrix([[5.4],[10.45]])

# Aw = matrixWidget(3,3)
# Aw.setM(A)
# Bw = matrixWidget(3,1)
# Bw.setM(B)
# Cw = matrixWidget(1,3)
# Cw.setM(C)
X0w = matrixWidget(2,1)
X0w.setM(X0)
Kw = matrixWidget(1,3)
Kw.setM(K)
Lw = matrixWidget(2,1)
Lw.setM(L)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig3c = matrixWidget(1,1)
eig1c.setM(numpy.matrix([-0.06])) 
eig2c.setM(numpy.matrix([[-2.43],[-2.41]]))
eig3c.setM(numpy.matrix([-0.06]))

eig1o = matrixWidget(1,1)
eig2o = matrixWidget(2,1)
eig1o.setM(numpy.matrix([-10.])) 
eig2o.setM(numpy.matrix([[-10.],[0.]]))

In [16]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= ['Nastavi K in L', 'Nastavi lastne vrednosti'],
    value= 'Nastavi lastne vrednosti',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues
sele = widgets.Dropdown(
    options= ['brez kompleksnih lastnih vrednosti', 'dve kompleksni lastni vrednosti'],
    value= 'dve kompleksni lastni vrednosti',
    description='Kompleksne lastne vrednosti:',
    style = {'description_width': 'initial'},
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=['impulzna funkcija', 'koračna funkcija', 'sinusoidna funkcija', 'kvadratni val'],
    value='koračna funkcija',
    description='Vhod:',
    style = {'description_width': 'initial'},
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=30.,
    min=1.,
    max=150.0,
    step=1,
    description='Referenca:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=30,
    min=1,
    max=80,
    step=1,
    description='Perioda: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
time = widgets.BoundedIntText(
    value=120,
    min=1,
    max=200,
    step=1,
    description='Čas:',
    disabled=False
)
sim = widgets.Checkbox(
    value=False,
    description='Nasičenje',
    disabled=False,
    indent=False
)

In [17]:
# Support functions

def eigen_choice(sele):
    if sele == 'brez kompleksnih lastnih vrednosti':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eig1o.children[0].children[0].disabled = False
        eig2o.children[1].children[0].disabled = True
        eig = 0
    if sele == 'dve kompleksni lastni vrednosti':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eig1o.children[0].children[0].disabled = True
        eig2o.children[1].children[0].disabled = False
        eig = 2
    return eig

def method_choice(selm):
    if selm == 'Nastavi K in L':
        method = 1
        sele.disabled = True
    if selm == 'Nastavi lastne vrednosti':
        method = 2
        sele.disabled = False
    return method

def simulation(sys_G, sys_K, sys_int, x0, x0k, time, Ts, ref, lim):
    '''Function that simulate a siso feedback system with a saturation in the actuator and integral action.
       Input to sys_K [u_sys_G; y_sys_G].
       sys_int must have k_3 as gain in output and B = [1, -1] associated to y and ref.'''
    if len(time) != len(ref):
        print('Napaka: T in ref morata biti enake velikosti!')
        return

    # Initialization and preparation...
    K_temp = control.append(sys_K,sys_int)
    K = sss(K_temp.A, 
            K_temp.B*sym.Matrix([[1,0,0],[0,1,0],[0,1,0],[0,0,1]]), 
            sym.Matrix([1,1]).T*K_temp.C, 
            sym.Matrix([1,1]).T*K_temp.D*sym.Matrix([[1,0,0],[0,1,0],[0,1,0],[0,0,1]]))
    sys_temp = control.append(sys_G,K)
    sys_temp1 = sss(sys_temp.A, 
                    sys_temp.B*sym.Matrix([[1,0,0],[1,0,0],[0,1,0],[0,0,1]]),
                    sys_temp.C,
                    sys_temp.D*sym.Matrix([[1,0,0],[1,0,0],[0,1,0],[0,0,1]]))
    #sys = control.connect(sys_temp1,[[2,1]],[1,3],[1,2])
    sys = sss(sys_temp1.A+sys_temp1.B[:,1]*sys_temp1.C[0,:],
              sym.Matrix(sys_temp1.B[:,0]).row_join(sym.Matrix(sys_temp1.B[:,2])),
              sys_temp1.C,
              [[0,0],[0,0]])
    sys_d = control.c2d(sys,Ts)
    A, B, C, D = sys_d.A, sys_d.B, sys_d.C, sys_d.D

    # Initializing system's variables
    x = [x0.col_join(x0k).col_join(sym.Matrix([0])) for t in range(0,len(time))]
    yinit = C*x[0]
    y = [yinit for t in range(0,len(time))]
    Input = [sym.Matrix([yinit[1],ref[t]]) for t in range(0,len(time))]

    for i in range(1,len(time)):
        Input[i-1][0] = -y[i-1][1]
        if Input[i-1][0] > lim:
            Input[i-1][0] = lim
        elif Input[i-1][0] < 0:
            Input[i-1][0] = 0.
    
        x[i] = A*x[i-1] + B*Input[i-1]
        y[i] = C*x[i-1] + D*Input[i-1]
        
    if Input[-1][0] > lim:
            Input[-1][0] = lim
    elif Input[-1][0] < 0:
            Input[-1][0] = 0.
            
    y  = [y[t][0] for t in range(len(y))]
    U  = [Input[t][0] for t in range(len(Input))]
    x  = [[x[i][t] for i in range(len(x))] for t in range(A.shape[0])]
    return (y, U, x)

def simulation_ideal(sys_G, sys_K, sys_int):
    K_temp = control.append(sys_K,sys_int)
    Ka = sss(K_temp.A, 
            K_temp.B*sym.Matrix([[1,0,0],[0,1,0],[0,1,0],[0,0,1]]), 
            sym.Matrix([1,1]).T*K_temp.C, 
            sym.Matrix([1,1]).T*K_temp.D*sym.Matrix([[1,0,0],[0,1,0],[0,1,0],[0,0,1]]))
    sys_temp = control.append(sys_G,Ka)
    sys_temp1 = sss(sys_temp.A, 
                    sys_temp.B*sym.Matrix([[1,0,0],[1,0,0],[0,1,0],[0,0,1]]),
                    sys_temp.C,
                    sys_temp.D*sym.Matrix([[1,0,0],[1,0,0],[0,1,0],[0,0,1]]))
    #sys = control.connect(sys_temp1,[[2,1],[1,-2]],[3],[1,2])
    sys = sss(sys_temp1.A+sys_temp1.B[:,1]*sys_temp1.C[0,:]-sys_temp1.B[:,0]*sys_temp1.C[1,:],
              sys_temp1.B[:,2],
              sys_temp1.C,
              [[0],[0]])
    return sys

In [18]:
sols = numpy.linalg.eig(A)
x0 = sym.Matrix([0,0])
lim = 1.

def main_callback(X0w, K, L, eig1c, eig2c, eig3c, eig1o, eig2o, u, period, selm, sele, selu, sim, time, DW):
    eige = eigen_choice(sele)
    method = method_choice(selm)
    
    if method == 1:
        solc = numpy.linalg.eig(Aa-Ba*K)
        solo = numpy.linalg.eig(A-L*C)
    if method == 2:
        if eige == 0:
            K = control.acker(Aa, Ba, [eig1c[0,0], eig2c[0,0], eig3c[0,0]])
            Kw.setM(K)
            L = control.acker(A.T, C.T, [eig1o[0,0], eig2o[0,0]]).T
            Lw.setM(L)
        if eige == 2:
            K = control.acker(Aa, Ba, [eig3c[0,0], 
                                     numpy.complex(eig2c[0,0],eig2c[1,0]), 
                                     numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
            L = control.acker(A.T, C.T, [numpy.complex(eig2o[0,0],eig2o[1,0]), 
                                         numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)

    # Simulation without saturation
    sys_G   = sss(A,B,C,0)
    sys_K   = sss(A-L*C,sym.Matrix(B).row_join(sym.Matrix(L)),K[0][0,:2],[0,0])
    sys_int = sss(0,sym.Matrix([1,-1]).T,K[0][0,2],[0,0])
    x0 = sym.Matrix([0.,0.])
    sys = simulation_ideal(sys_G, sys_K, sys_int)
    solc = sym.Matrix(sys.A).eigenvals(multiple=True,rational=False)
    solc = numpy.matrix(solc,dtype=numpy.complex128)
    print('Lastni vrednosti sistema sta:', round(sols[0][0],2),'and', round(sols[0][1],2))
    print('Lastne vrednosti krmiljenega zaprtozančnega sistema so:', 
          round(solc[0,0],2),',', 
          round(solc[0,1],2),',', 
          round(solc[0,2],2),',',
          round(solc[0,3],2),'in',
          round(solc[0,4],2))
    
    X0w1 = [float(x0[0,0]),float(x0[1,0]),float(X0w[0,0]),float(X0w[1,0]),0.]
    T, Td = numpy.linspace(0,time,1000,retstep=True)
      
    if selu == 'impulzna funkcija': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        if sim == False:
            T, yout, xout = control.forced_response(sys,T,U,X0w1)
    if selu == 'koračna funkcija':
        U = [u for t in range(0,len(T))]
        if sim == False:
            T, yout, xout = control.forced_response(sys,T,U,X0w1)
    if selu == 'sinusoidna funkcija':
        U = u*numpy.sin(2*numpy.pi/period*T)+u
        if sim == False:
            T, yout, xout = control.forced_response(sys,T,U,X0w1)
    if selu == 'kvadratni val':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))+u
        if sim == False:
            T, yout, xout = control.forced_response(sys,T,U,X0w1)
    
    if sim:
        y_sat, U_sat, x_sat = simulation(sys_G, sys_K, sys_int, x0, sym.Matrix(X0w), T, Td, U, lim)
    
    fig = plt.figure(num='Simulacija', figsize=(14,12))
    fig.add_subplot(224)
    plt.title('Odziv sistema')
    plt.ylabel('Izhod')
    if sim:
        plt.plot(T,y_sat,T,U,'r--')
        plt.legend(['$y_{sat}$','Referenca'])
    else:
        plt.plot(T,yout[0],T,U,'r--')
        plt.legend(['$y$','Referenca'])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(221)
    plt.title('Odziv prvega stanja')
    plt.ylabel('$x_1$')
    if sim:
        plt.plot(T,x_sat[0],T,x_sat[2])
        plt.legend(['$x_{1sat}$','$x_{1est}$'])
    else:
        plt.plot(T,xout[0],T,xout[2])
        plt.legend(['$x_{1}$','$x_{1est}$'])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(222)
    plt.title('Odziv drugega stanja')
    plt.ylabel('$x_2$')
    if sim:
        plt.plot(T,x_sat[1],T,x_sat[3])
        plt.legend(['$x_{2sat}$','$x_{2est}$'])
    else:
        plt.plot(T,xout[1],T,xout[3])
        plt.legend(['$x_{2}$','$x_{2est}$'])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(223)
    plt.title('Vhod')
    plt.ylabel('Vhod')
    if sim:
        plt.plot(T[:-1],U_sat[:-1],T,[1 for t in range(len(T))],'r--')
        plt.legend(['dejanska vrednost','maks. vrednost'])
    else:
        plt.plot(T,-yout[1],T,[1 for t in range(len(T))],'r--')
        plt.legend(['dejanska vrednost','maks. vrednost'])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    if selu == 'step':
        if sim:
            print('Maksimalen prenihaj znaša:',round((max(y_sat)-U[0])/U[0]*100,3),'percent')
        else:
            print('Maksimalen prenihaj znaša:',round((max(yout[0])-U[0])/U[0]*100,3),'percent')

   
alltogether = widgets.VBox([widgets.HBox([selm, 
                                          sele,
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('K:',border=3), Kw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Lastne vrednosti:',border=3), 
                                          eig1c, 
                                          eig2c, 
                                          eig3c,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0 est.:',border=3), X0w]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('L:',border=3), Lw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Lastne vrednosti:',border=3), 
                                          eig1o, 
                                          eig2o, 
                                          widgets.Label(' ',border=3),
                                          time]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([u, 
                                          period,
                                          widgets.Label(' ',border=3),widgets.Label(' ',border=3),sim]),
                            widgets.HBox([widgets.Label('Klikni za K, L ali lastne vrednosti ->',border=3),
                            START])])
out = widgets.interactive_output(main_callback, {'X0w':X0w, 'K':Kw, 'L':Lw,
                                                 'eig1c':eig1c, 'eig2c':eig2c, 'eig3c':eig3c, 
                                                 'eig1o':eig1o, 'eig2o':eig2o, 
                                                 'u':u, 'period':period, 'selm':selm, 'sele':sele, 'selu':selu,
                                                 'sim':sim, 'time':time, 'DW':DW})
out.layout.height = '840px'
display(out, alltogether)

Output(layout=Layout(height='840px'))